In [1]:
import numpy as np, keras
#from mr import mr
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Dropout, Bidirectional, TimeDistributed, Masking
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

/home/soham/.env/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

In [3]:
embedding_dim = 300

In [4]:
X_train = np.load('train_x.npy'); y_train = np.load('train_y.npy'); l_train = np.load('train_l.npy')
X_valid = np.load('valid_x.npy'); y_valid = np.load('valid_y.npy'); l_valid = np.load('valid_l.npy')
X_test  = np.load('test_x.npy' ); y_test  = np.load('test_y.npy' ); l_test  = np.load('test_l.npy' )

In [5]:
vocab_size = np.max(list(set.union(*[set(x) for x in X_train]))) + 1

In [6]:
max_length = len(X_train[0])

In [7]:
dictionary_fwd = np.load('dictionary_fwd.npy').item()

In [8]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [9]:
# create the model
model = Sequential()
model.add(Masking(mask_value=0, input_shape=(127,)))
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length, \
                    weights=[embedding_matrix], trainable=True))
model.add(Bidirectional(GRU(150, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)))
model.add(Bidirectional(GRU(150, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)))
model.add(Bidirectional(GRU(150, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)))
model.add(TimeDistributed(Dense(3, activation='softmax')))

In [10]:
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 127)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 127, 300)          3336000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 127, 300)          405900    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 127, 300)          405900    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 127, 300)          405900    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 127, 3)            903       
Total params: 4,554,603
Trainable params: 4,554,603
Non-trainable params: 0
_________________________________________________________________


In [11]:
from keras.callbacks import ModelCheckpoint

In [12]:
mcp = ModelCheckpoint('bidir_rnn_scratch.hdf5', monitor="val_acc", save_best_only=True, save_weights_only=False)

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [14]:
y_train2 = np.array([to_categorical(y, 3) for y in y_train])
y_valid2 = np.array([to_categorical(y, 3) for y in y_valid])
y_test2 = np.array([to_categorical(y, 3) for y in y_test])

In [15]:
model.fit(X_train, y_train2, epochs=30, batch_size=128, validation_data=(X_valid, y_valid2), callbacks=[mcp])

Train on 2558 samples, validate on 365 samples
Epoch 1/30
2558/2558 [==============================] - 28s - loss: 0.1722 - acc: 0.9818 - val_loss: 0.0565 - val_acc: 0.9816
Epoch 2/30
2558/2558 [==============================] - 26s - loss: 0.0466 - acc: 0.9823 - val_loss: 0.0519 - val_acc: 0.9823
Epoch 3/30
2558/2558 [==============================] - 26s - loss: 0.0373 - acc: 0.9850 - val_loss: 0.0482 - val_acc: 0.9841
Epoch 4/30
2558/2558 [==============================] - 26s - loss: 0.0285 - acc: 0.9888 - val_loss: 0.0440 - val_acc: 0.9874
Epoch 5/30
2558/2558 [==============================] - 26s - loss: 0.0201 - acc: 0.9926 - val_loss: 0.0392 - val_acc: 0.9885
Epoch 6/30
2558/2558 [==============================] - 26s - loss: 0.0141 - acc: 0.9949 - val_loss: 0.0409 - val_acc: 0.9887
Epoch 7/30
2558/2558 [==============================] - 26s - loss: 0.0125 - acc: 0.9956 - val_loss: 0.0434 - val_acc: 0.9891
Epoch 8/30
2558/2558 [==============================] - 26s - loss: 0.0

In [20]:
model.load_weights('bidir_rnn_scratch.hdf5')

In [21]:
result = model.evaluate(X_test, y_test2, batch_size=128)

732/732 [==============================] - 1s     


In [22]:
predictions = model.predict(X_test)

In [23]:
with open('predictions_rnn_scratch.txt', 'w') as f:
    for tokens, prediction in zip(X_test, predictions):
        started = False

        for tok, pred in zip(tokens, prediction):
            if not started and tok == 1:
                started = True
            elif started and tok == 2:
                f.write('\n')
                break
            elif started:
                if np.argmax(pred) == 0:
                    f.write('O\n')
                elif np.argmax(pred) == 1:
                    f.write('T\n')
                elif np.argmax(pred) == 2:
                    f.write('D\n')